In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [2]:
burns=pd.read_csv("data2/endocrine.csv")

In [3]:
apachepredvar = pd.read_csv('apachePredVar.csv.gz')

In [4]:
burns_merged = pd.merge(apachepredvar[['patientunitstayid','pao2','fio2']], burns, how='right', on='patientunitstayid')

In [5]:
burns_merged['lods_ratio'] = 0

In [6]:
for i in range(len(burns_merged)):
    if (burns_merged.iloc[i]['pao2'] != -1) & (burns_merged.iloc[i]['fio2'] != -1) & (burns_merged.iloc[i]['ventday1'] == 1):
        if ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 < 150):
            burns_merged['lods_ratio'].iloc[i] = 3
        elif ((burns_merged.iloc[i]['pao2']/burns_merged.iloc[i]['fio2']) * 100 >= 150):
            burns_merged['lods_ratio'].iloc[i] = 1

C:\Users\nsafa\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [7]:
def platelets(item):
    temp = 0
    if (item < 50):
        temp += 1
    return temp

In [8]:
burns_merged['lods_platelets'] = burns_merged['platelets x 1000'].apply(platelets)

In [9]:
burns_merged['GCS'] = burns_merged['eyes'] + burns_merged['motor'] + burns_merged['verbal']

In [10]:
def GCS(item):
    temp = 0
    if (item >= 9) & (item <= 13):
        temp += 1
    elif (6 <= item) & (item <= 8):
        temp += 3
    elif (3 <= item) & (item <= 5):
        temp += 5
    return temp

In [11]:
burns_merged['lods_GCS'] = burns_merged['GCS'].apply(GCS)

In [12]:
apacheapsdvar = pd.read_csv('ApacheApsVar.csv.gz')

In [13]:
burns_merged_1 = pd.merge(apacheapsdvar[['patientunitstayid','bilirubin']], burns_merged, how='right', on='patientunitstayid')

In [14]:
def bilirubin(item):
    temp = 0
    if (item >= 2.0):
        temp += 1
    return temp

In [15]:
burns_merged_1['lods_bilirubin'] = burns_merged_1['bilirubin'].apply(bilirubin)

In [16]:
def meanbp(item):
    temp = 0
    if (item >= 70) & (item < 90):
        temp += 1
    if (item >= 240) & (item < 270):
        temp += 1 
    if (item >= 40) & (item < 70):
        temp += 3
    if (item >= 270):
        temp += 3
    if (item < 40):
        temp += 5
    return temp

In [17]:
burns_merged_1['lods_meanbp'] = burns_merged_1['meanbp'].apply(meanbp)

In [18]:
def creatinine(item):
    temp = 0
    if (item >= 1.2) & (item <= 1.6):
        temp += 1
    if (item > 1.6):
        temp += 3
    return temp

In [19]:
burns_merged_1['lods_creatinine'] = burns_merged_1['creatinine'].apply(creatinine)

In [20]:
burns_merged_2 = pd.merge(apacheapsdvar[['patientunitstayid','urine']], burns_merged_1, how='right', on='patientunitstayid')

In [21]:
burns_merged_2['urine_L'] = burns_merged_2['urine']/1000

In [22]:
def urine(item):
    temp = 0
    if (item >= 0.5) & (item <= 0.75):
        temp += 3
    if (item >= 10):
        temp += 3
    if (item < 0.5):
        temp += 5
    return temp

In [23]:
burns_merged_2['lods_urine'] = burns_merged_2['urine_L'].apply(urine)

In [24]:
def platelets(item):
    temp = 0
    if (item < 50):
        temp += 1
    return temp

In [25]:
burns_merged_2['lods_platelets'] = burns_merged_2['platelets x 1000'].apply(platelets)

In [26]:
def heartrate(item):
    temp = 0
    if (item >= 140):
        temp += 1
    elif (item < 30):
        temp += 5
    return temp

In [27]:
burns_merged_2['lods_heartrate'] = burns_merged_2['heartrate'].apply(heartrate)

In [28]:
def BUN(item):
    temp = 0
    if (item >= 16.8) & (item < 28):
        temp += 1
    if (item >= 28) & (item < 56):
        temp += 3
    if (item >= 56):
        temp += 5
    return temp

In [29]:
burns_merged_2['lods_BUN'] = burns_merged_2['BUN'].apply(BUN)

In [30]:
def WBC(item):
    temp = 0
    if (item >= 1) & (item < 2.5):
        temp += 1
    if (item < 1):
        temp += 3
    if (item >= 50):
        temp += 1
    return temp

In [31]:
burns_merged_2['lods_wbc'] = burns_merged_2['WBC x 1000'].apply(WBC)

In [32]:
 def LODS_mort(item):
    
    return np.exp(-3.4043 + 0.4173*(item))/(1+np.exp(-3.4043 + 0.4173*(item)))

In [33]:
burns_merged_2['LODS'] = burns_merged_2[['lods_ratio', 'lods_platelets', 'lods_GCS',
       'lods_bilirubin', 'lods_meanbp', 'lods_creatinine',
       'lods_urine', 'lods_heartrate', 'lods_BUN', 'lods_wbc']].sum(axis=1)

In [34]:
burns_merged_2['LODS_mort'] = burns_merged_2['LODS'].apply(LODS_mort)

In [35]:
burns_merged_2['actualicumortality_label'] = burns_merged_2['actualicumortality'].apply(lambda x: 1 if x =='EXPIRED' else 0)

In [36]:
fpr, tpr, thresholds = metrics.roc_curve(burns_merged_2['actualicumortality_label'].values, burns_merged_2['LODS_mort'].values, pos_label=1)

In [37]:
metrics.auc(fpr, tpr)

0.8043653851750899

In [38]:
burns['patientunitstayid'].nunique()

13667

In [39]:
burns_merged_2['actualicumortality_label'].sum()

605

In [40]:
burns_merged_2['actualicumortality_label'].sum()/burns['patientunitstayid'].nunique()

0.044267212994805